In [1]:
# モジュールインポート
import matplotlib.pyplot as plt
import matplotlib.pylab as plab
import numpy as np
import math
import sys
import os
import pickle
import pprint
from PIL import Image
from mpl_toolkits.mplot3d import Axes3D

# dataset.mnistをインポートするために親ディレクトリをpathに追加
sys.path.append(os.pardir)
from dataset.mnist import load_mnist

In [2]:
# ライブラリ化した活性化関数のインポート

#　シグモイド関数
def sigmoid(x):
    return 1/(1 + np.exp(-1*x))

#　ステップ関数
def step_function(x):
    y = x > 0
    return y.astype(np.int)

#　ReLU関数
def ReLU(x):
    return np.maximum(0, x)

#　ソフトマックス関数
def softmax(x):
    x = x - np.max(x, axis = -1, keepdims = True)
    return np.exp(x)/np.sum(np.exp(x), axis = -1, keepdims = True)

In [3]:
# 損失関数のインポート

# 二乗和誤差
def mean_squared_error(y, t):
    
    return 0.5*(np.sum((y-t)**2))

# 交差エントロピー
def cross_entropy_error(y, t):
    
    if y.ndim == 1:
        # 1行の2次元配列を生成
        # データが複数ある場合と1つしかない場合でこの後の処理を共有するため
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # one-hot-label の場合
    if y.size == t.size:
        # 1があるlabelに変換
        t = t.argmax(axis = 1)
    
    # batch_size = データ数
    batch_size = y.shape[0]

    # 桁落ち防止
    delta = 1e-7
    # ベクトルyの中で、教師ラベルのデータ位置tだけ計算して交差エントロピーを計算
    # y[2, 4]とかになる、これは3個目のデータの4番目の値となる
    e = -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size
    
    return e

In [4]:
#　数値微分(多次元ベクトル対応版)
def numerical_gradient(f, x):
    
    h = 1e-4
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags = ["multi_index"], op_flags = ["readwrite"])
    # np.nditerによって多次元配列でも各要素ごとにループできる
    while not it.finished:
        idx = it.multi_index
        tmp = x[idx]
        
        # f(x+h)
        x[idx] = tmp + h
        f_plus_h = f(x)
        
        # f(x+h)
        x[idx] = tmp - h
        f_minus_h = f(x)
        
        # diff
        grad[idx] = (f_plus_h - f_minus_h)/(2*h)
        x[idx] = tmp
        
        it.iternext()
        
    return grad